In [2]:
import cv2
import mediapipe as mp
import time
import numpy as np
import pyautogui

In [3]:
time.sleep(2.0)

mp_draw=mp.solutions.drawing_utils
mp_hand=mp.solutions.hands

pyautogui.FAILSAFE = False
tipIds=[4,8,12,16,20]
wc,hc=640,480
video=cv2.VideoCapture(0)
video.set(3,wc)
video.set(4,hc)
framer=100
ws,hs=1920,1080
clx,cly=0,0
plx,ply=0,0
ptime=0
smooth=4
with mp_hand.Hands(min_detection_confidence=0.5,
               min_tracking_confidence=0.5) as hands:
    while True:
        
        ret,image=video.read()
        image=cv2.flip(image,1)
        image=cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image.flags.writeable=False
        results=hands.process(image)
        image.flags.writeable=True
        image=cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        lmList=[]
        xlist=[]
        ylist=[]
        if results.multi_hand_landmarks:
            for hand_landmark in results.multi_hand_landmarks: #for multi hand detect
                myHands=results.multi_hand_landmarks[0]
                for id, lm in enumerate(myHands.landmark):
                    h,w,c=image.shape
                    cx,cy= int(lm.x*w), int(lm.y*h)
                    xlist.append(cx)
                    ylist.append(cy)
                    lmList.append([id,cx,cy])
                mp_draw.draw_landmarks(image, hand_landmark, mp_hand.HAND_CONNECTIONS)
        fingers=[]
        cv2.rectangle(image, (framer, framer), (wc-framer, hc-framer), (0, 255, 0), 2)
        if len(lmList)!=0:
            x1,y1=lmList[8][1:]
            x2,y2=lmList[12][1:]
            if lmList[tipIds[0]][1] > lmList[tipIds[0]-1][1]:
                fingers.append(1)
            else:
                fingers.append(0)
            for id in range(1,5):
                if lmList[tipIds[id]][2] < lmList[tipIds[id]-2][2]:
                    fingers.append(1)
                else:
                    fingers.append(0)
            total=fingers.count(1)
            xmin,xmax=min(xlist),max(xlist)
            ymin,ymax=min(ylist),max(ylist)
            bbox=xmin,xmax,ymin,ymax
            cv2.rectangle(image, (xmin-20, ymin-20), (xmax+20, ymax+20), (0, 255, 0), 2) 
            if fingers[1]==1 and fingers[2]==0:
                x3=np.interp(x1,(framer,wc-framer),(0,ws))
                y3=np.interp(y1,(framer,hc-framer),(0,hs))
                clx=plx+ (x3-plx)/smooth
                cly=ply+(y3-ply)/smooth
                pyautogui.moveTo(clx,cly)
                cv2.circle(image, (x1,y1),15, (0, 255, 0), cv2.FILLED)
                plx,ply=clx,cly
            if fingers[1]==1 and fingers[2]==1:
                x3=np.interp(x2,(framer,wc-framer),(0,ws))
                y3=np.interp(y2,(framer,hc-framer),(0,hs))
               
                pyautogui.click()
                cv2.circle(image, (x1+40,y1-10),15, (0, 255, 0), cv2.FILLED)
                

        ctime=time.time()
        fps=1/(ctime-ptime)
        ptime=ctime
        
        cv2.putText(image, str(int(fps)), (45, 375), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 0, 0), 2)
        cv2.imshow("Frame",image)
        k=cv2.waitKey(1)
        if k==ord('q'):
            break
video.release()
cv2.destroyAllWindows()




C:\ProgramData\Anaconda3\lib\site-packages\google\protobuf\symbol_database.py:78: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
